In [ ]:
import snapista
from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt
from getpass import getpass
from shapely.geometry import box
from snapista import Graph
from snapista import Operator
from snapista import TargetBand
from snapista import TargetBandDescriptors

## Data access configuration

In [ ]:

user = getpass('scihub username') 
password = getpass('scihub password') 

api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/dhus')

## Sentinel-2 discovery

In [ ]:

aoi = [14.716,37.603,15.109,37.897]


search_terms = dict()

search_terms['area'] = box(*aoi).wkt
search_terms['date'] = ('2021-05-22T00:00:00Z', '2021-05-22T23:59:59Z')
search_terms['platformname'] = 'Sentinel-2'
search_terms['processinglevel'] = 'Level-1C'
search_terms['cloudcoverpercentage'] = (0, 20)

search_terms

In [ ]:
s2_products = api.to_geodataframe(api.query(**search_terms))

s2_products

In [ ]:
s2_products['aoi_intersec'] = s2_products.apply(lambda row: get_intersection(row, 
                                                                                  box(*aoi)), 
                                              axis=1)

In [ ]:
s2_products

## Sentinel-1 discovery

In [ ]:
aoi = [14.716,37.603,15.109,37.897]


search_terms = dict()

search_terms['area'] = box(*aoi).wkt
search_terms['date'] = ('2021-06-03T00:00:00Z', '2021-06-03T23:59:59Z')
search_terms['platformname'] = 'Sentinel-1'
search_terms['producttype'] = 'GRD'


search_terms

In [ ]:
s1_products = api.to_geodataframe(api.query(**search_terms))

s1_products

In [ ]:
def get_intersection(row, aoi):

        return (row['geometry'].intersection(aoi).area / aoi.area) * 100

In [ ]:
s1_products['aoi_intersec'] = s1_products.apply(lambda row: get_intersection(row, 
                                                                                  box(*aoi)), 
                                              axis=1)

In [ ]:
s1_products

In [ ]:
s1_products[s1_products.aoi_intersec > 90.0].iloc[0].name

In [ ]:
import pandas as pd
products = pd.concat([s1_products[s1_products.aoi_intersec > 90.0], s2_products[s2_products.aoi_intersec > 90.0]])

In [ ]:
list(products.index.values)

In [ ]:
for prd in list(products.index.values):
    api.download(prd)

In [ ]:
api.download_all(list(products.index.values), n_concurrent_dl=1)

In [ ]:
api.download(s1_products[s1_products.aoi_intersec > 90.0].iloc[0].name)

```xml
<graph id="Graph">
  <version>1.0</version>
  <node id="Read_SAR">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <file>/home/fbrito/Downloads/collocation/S1A_IW_GRDH_1SDV_20210603T050456_20210603T050521_038171_04814B_3BC4.SAFE/manifest.safe</file>
      <formatName>SENTINEL-1</formatName>
    </parameters>
  </node>
  <node id="Read_optical">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <file>/home/fbrito/Downloads/collocation/S2B_MSIL2A_20210522T095029_N0300_R079_T33SVB_20210522T113428.SAFE/MTD_MSIL2A.xml</file>
      <formatName>SENTINEL-2-MSI-MultiRes-UTM33N</formatName>
    </parameters>
  </node>
  <node id="Calibration">
    <operator>Calibration</operator>
    <sources>
      <sourceProduct refid="Read_SAR"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <sourceBands/>
      <auxFile>Product Auxiliary File</auxFile>
      <externalAuxFile/>
      <outputImageInComplex>false</outputImageInComplex>
      <outputImageScaleInDb>false</outputImageScaleInDb>
      <createGammaBand>false</createGammaBand>
      <createBetaBand>false</createBetaBand>
      <selectedPolarisations/>
      <outputSigmaBand>true</outputSigmaBand>
      <outputGammaBand>false</outputGammaBand>
      <outputBetaBand>false</outputBetaBand>
    </parameters>
  </node>
  <node id="Speckle-Filter">
    <operator>Speckle-Filter</operator>
    <sources>
      <sourceProduct refid="Calibration"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <sourceBands/>
      <filter>Lee</filter>
      <filterSizeX>3</filterSizeX>
      <filterSizeY>3</filterSizeY>
      <dampingFactor>2</dampingFactor>
      <estimateENL>true</estimateENL>
      <enl>1.0</enl>
      <numLooksStr>1</numLooksStr>
      <windowSize>7x7</windowSize>
      <targetWindowSizeStr>3x3</targetWindowSizeStr>
      <sigmaStr>0.9</sigmaStr>
      <anSize>50</anSize>
    </parameters>
  </node>
  <node id="LinearToFromdB">
    <operator>LinearToFromdB</operator>
    <sources>
      <sourceProduct refid="Speckle-Filter"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <sourceBands/>
    </parameters>
  </node>
  <node id="Terrain-Correction">
    <operator>Terrain-Correction</operator>
    <sources>
      <sourceProduct refid="LinearToFromdB"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <sourceBands/>
      <demName>SRTM 1Sec HGT</demName>
      <externalDEMFile/>
      <externalDEMNoDataValue>0.0</externalDEMNoDataValue>
      <externalDEMApplyEGM>true</externalDEMApplyEGM>
      <demResamplingMethod>BILINEAR_INTERPOLATION</demResamplingMethod>
      <imgResamplingMethod>BILINEAR_INTERPOLATION</imgResamplingMethod>
      <pixelSpacingInMeter>10.0</pixelSpacingInMeter>
      <pixelSpacingInDegree>8.983152841195215E-5</pixelSpacingInDegree>
      <mapProjection>PROJCS[&quot;UTM Zone 32 / World Geodetic System 1984&quot;, 
  GEOGCS[&quot;World Geodetic System 1984&quot;, 
    DATUM[&quot;World Geodetic System 1984&quot;, 
      SPHEROID[&quot;WGS 84&quot;, 6378137.0, 298.257223563, AUTHORITY[&quot;EPSG&quot;,&quot;7030&quot;]], 
      AUTHORITY[&quot;EPSG&quot;,&quot;6326&quot;]], 
    PRIMEM[&quot;Greenwich&quot;, 0.0, AUTHORITY[&quot;EPSG&quot;,&quot;8901&quot;]], 
    UNIT[&quot;degree&quot;, 0.017453292519943295], 
    AXIS[&quot;Geodetic longitude&quot;, EAST], 
    AXIS[&quot;Geodetic latitude&quot;, NORTH]], 
  PROJECTION[&quot;Transverse_Mercator&quot;], 
  PARAMETER[&quot;central_meridian&quot;, 9.0], 
  PARAMETER[&quot;latitude_of_origin&quot;, 0.0], 
  PARAMETER[&quot;scale_factor&quot;, 0.9996], 
  PARAMETER[&quot;false_easting&quot;, 500000.0], 
  PARAMETER[&quot;false_northing&quot;, 0.0], 
  UNIT[&quot;m&quot;, 1.0], 
  AXIS[&quot;Easting&quot;, EAST], 
  AXIS[&quot;Northing&quot;, NORTH]]</mapProjection>
      <alignToStandardGrid>false</alignToStandardGrid>
      <standardGridOriginX>0.0</standardGridOriginX>
      <standardGridOriginY>0.0</standardGridOriginY>
      <nodataValueAtSea>false</nodataValueAtSea>
      <saveDEM>false</saveDEM>
      <saveLatLon>false</saveLatLon>
      <saveIncidenceAngleFromEllipsoid>false</saveIncidenceAngleFromEllipsoid>
      <saveLocalIncidenceAngle>false</saveLocalIncidenceAngle>
      <saveProjectedLocalIncidenceAngle>false</saveProjectedLocalIncidenceAngle>
      <saveSelectedSourceBand>true</saveSelectedSourceBand>
      <outputComplex>false</outputComplex>
      <applyRadiometricNormalization>false</applyRadiometricNormalization>
      <saveSigmaNought>false</saveSigmaNought>
      <saveGammaNought>false</saveGammaNought>
      <saveBetaNought>false</saveBetaNought>
      <incidenceAngleForSigma0>Use projected local incidence angle from DEM</incidenceAngleForSigma0>
      <incidenceAngleForGamma0>Use projected local incidence angle from DEM</incidenceAngleForGamma0>
      <auxFile>Latest Auxiliary File</auxFile>
      <externalAuxFile/>
    </parameters>
  </node>
  <node id="Collocate">
    <operator>Collocate</operator>
    <sources>
      <sourceProduct refid="Subset_WKT_SAR"/>
      <sourceProduct.1 refid="Subset_WKT_optical"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <sourceProductPaths/>
      <masterProductName>Subset_S1A_IW_GRDH_1SDV_20210603T050456_20210603T050521_038171_04814B_3BC4_Cal_Spk_dB_TC</masterProductName>
      <targetProductName>_collocated</targetProductName>
      <targetProductType>COLLOCATED</targetProductType>
      <renameMasterComponents>true</renameMasterComponents>
      <renameSlaveComponents>true</renameSlaveComponents>
      <masterComponentPattern>${ORIGINAL_NAME}_M</masterComponentPattern>
      <slaveComponentPattern>${ORIGINAL_NAME}_S${SLAVE_NUMBER_ID}</slaveComponentPattern>
      <resamplingType>NEAREST_NEIGHBOUR</resamplingType>
    </parameters>
  </node>
  <node id="Subset_WKT_SAR">
    <operator>Subset</operator>
    <sources>
      <sourceProduct refid="Terrain-Correction"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <sourceBands/>
      <region></region>
      <referenceBand/>
      <geoRegion>POLYGON((14.716 37.603,14.716 37.897,15.109 37.897,15.109 37.603,14.716 37.603))</geoRegion>
      <subSamplingX>1</subSamplingX>
      <subSamplingY>1</subSamplingY>
      <fullSwath>false</fullSwath>
      <tiePointGridNames/>
      <copyMetadata>true</copyMetadata>
    </parameters>
  </node>
  <node id="Subset_WKT_optical">
    <operator>Subset</operator>
    <sources>
      <sourceProduct refid="Read_optical"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <sourceBands/>
      <region></region>
      <referenceBand/>
      <geoRegion>POLYGON((14.716 37.603,14.716 37.897,15.109 37.897,15.109 37.603,14.716 37.603))</geoRegion>
      <subSamplingX>1</subSamplingX>
      <subSamplingY>1</subSamplingY>
      <fullSwath>false</fullSwath>
      <tiePointGridNames/>
      <copyMetadata>true</copyMetadata>
    </parameters>
  </node>
  <node id="Write">
    <operator>Write</operator>
    <sources>
      <sourceProduct refid="Collocate"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <file>/home/fbrito/S1S2_collocated.dim</file>
      <formatName>BEAM-DIMAP</formatName>
    </parameters>
  </node>
  <applicationData id="Presentation">
    <Description/>
    <node id="Read_SAR">
            <displayPosition x="9.0" y="19.0"/>
    </node>
    <node id="Read_optical">
      <displayPosition x="10.0" y="234.0"/>
    </node>
    <node id="Calibration">
      <displayPosition x="8.0" y="69.0"/>
    </node>
    <node id="Speckle-Filter">
      <displayPosition x="103.0" y="70.0"/>
    </node>
    <node id="LinearToFromdB">
      <displayPosition x="95.0" y="118.0"/>
    </node>
    <node id="Terrain-Correction">
      <displayPosition x="263.0" y="118.0"/>
    </node>
    <node id="Collocate">
      <displayPosition x="290.0" y="234.0"/>
    </node>
    <node id="Subset_WKT_SAR">
      <displayPosition x="266.0" y="175.0"/>
    </node>
    <node id="Subset_WKT_optical">
      <displayPosition x="138.0" y="234.0"/>
    </node>
    <node id="Write">
      <displayPosition x="411.0" y="235.0"/>
    </node>
  </applicationData>
</graph>
```

## SNAP Sentinel-1/Sentinel-2 collocation graph

In [ ]:
g = Graph()

In [ ]:
g.add_node(
    operator=Operator(
        "Read",
        formatName="SENTINEL-1",
        file="path_to_manifest",
    ),
    node_id="read_sar",
)

In [ ]:
g.add_node(
    operator=Operator(
        "Read",
        formatName="SENTINEL-2-MSI-MultiRes-UTM33N",
        file="path_to_manifest",
    ),
    node_id="read_optical",
)

In [ ]:
g.add_node(
    operator=Operator(
        "Calibration",
    ),
    node_id="calibration",
    source="read_sar"
)

In [ ]:


g.add_node(
    operator=Operator(
        "Speckle-Filter",
    ),
    node_id="speckle-filter",
    source="calibration"
)

In [ ]:
g.add_node(
    operator=Operator(
        "Terrain-Correction",
        demName="SRTM 1Sec HGT",
        mapProjection="EPSG:32633"
    ), 
    node_id="terrain-correction",
    source="speckle-filter"
    )

In [ ]:

g.add_node(
    operator=Operator(
        "Subset",
        geoRegion=box(*aoi).wkt,
        copyMetadata="true"
    ),
    node_id="subset_sar",
    source="terrain-correction"
)

In [ ]:


g.add_node(
    operator=Operator(
                "Subset",
        geoRegion=box(*aoi).wkt,
        copyMetadata="true"
    ),
    node_id="subset_optical",
    source="read_optical"
)

In [ ]:
#Collocate

g.add_node(
    operator=Operator(
                "Collocate",
             masterProductName="subset_S1A_IW_GRDH_1SDV_20210603T050456_20210603T050521_038171_04814B_3BC4_Cal_Spk_dB_TC",
        targetProductName="_collocated", 
        targetProductType="COLLOCATED",
        renameMasterComponents="true",
        renameSlaveComponents="true",
        masterComponentPattern="${ORIGINAL_NAME}_M",
        slaveComponentPattern="${ORIGINAL_NAME}_S${SLAVE_NUMBER_ID}",
        resamplingType="NEAREST_NEIGHBOUR",
        ),
    node_id="collocate",
    source=["subset_sar", "subset_optical"]
)

In [ ]:
g.add_node(
    operator=Operator("Write", file='result.dim', formatName="BEAM-DIMAP"),
    node_id="write",
    source="collocate",
)

In [ ]:
g.view()

In [ ]:
g.save_graph('collocation.xml')